In [1]:
# 1. x, t입력
# x : 소리
# 소리를 mfcc 때리면 (소리 길이, 13) 의 자료로 변환
# t : 과일 종류 7개

# 2. 함수 만들면
# Encoder = rnn(13, 13)
# Decoder = Linear(13, 7), decoder의 입력값은, Encoder의 마지막 값 (h)
# 만약 Encoder가 Bidirection = True 면
# h[0:1,:,:]
# h[1:2,:,:] 을 concat으로 이어 붙인 후, reshape 하고 (-1, 26)
# Decoder 입력값도 26개로 늘어남

# 손실함수 : CrossEntropyLoss

In [2]:
from scipy.io import wavfile #소리 불러오는 모듈
import numpy #소리 가공
from python_speech_features import mfcc #mfcc 계산해주는 함수

import torch #AI 모듈
import torch.nn as nn

import os

In [3]:
# 자료 불러오고 전처리 작업

#t 값 가져오기
t_list = os.listdir('data/')
remove_list = []

for t in t_list :
    if t.find('.') >= 0 :
        remove_list.append(t) 

for r in remove_list :
    t_list.remove(r)

print(t_list)
print(t_list.index('kiwi')) #해당 원소가 들어있는 위치 출력하는 함수

['apple', 'banana', 'kiwi', 'lime', 'orange', 'peach', 'pineapple']
2


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

data = []

#x값 가져오기
for t in t_list :
    x_list = os.listdir('data/' + t)
    # print(x_list)
    for x in x_list :
        if x.find('.wav') < 0 : #예외 : .wav 파일이 아니면
            continue
        freq, signal = wavfile.read('data/' + t + '/' + x) # 소리 읽어오는 부분
        x_data = mfcc(signal, freq) #numpy.array 형태의 데이터
        # print(x_data.shape)
        x_tensor = torch.tensor(x_data, dtype = torch.float, device = device).unsqueeze(0) #AI에서 계산하기 위해 바로 텐서로 변환, 3차원으로 만들기 위해 unsqueeze
        t_tensor = torch.tensor(t_list.index(t), dtype = torch.long, device = device)
        temp = (x_tensor, t_tensor) #x값과 t값을 tuple 형태로 묶음, Dataloader를 수동으로 만드는 것과 비슷함
        data.append(temp)
        # print(data)

print(len(data))
print(data[10][0].shape)

#하나하나씩 데이터를 넣으면 padding작업이 필요없다 (padding은 길이가 다른 자료들을 한번에 계산하기위해 하는 것)

105
torch.Size([1, 43, 13])


In [5]:
from NN import Encoder
from NN import Decoder
from NN import Encoder_n_Decoder

In [6]:
encoder = Encoder(13)
decoder = Decoder(26)
loss_function = nn.CrossEntropyLoss() #분류분제 (네, 아니오)
encoder_optim = torch.optim.Adam(encoder.parameters(), lr = 0.002)
decoder_optim = torch.optim.Adam(decoder.parameters(), lr = 0.002)
epoch = 20

for e in range(epoch) :
    loss_sum = 0
    encoder.train() #dropout 켜기
    decoder.train()
    for x, t in data :
        y, hc = encoder(x)
        y = decoder(hc[0])

        loss = loss_function(y, t.unsqueeze(0)) #y는 2차원, t는 1차원
        loss_sum += loss.item()

        loss.backward()
        decoder_optim.step()
        encoder_optim.step()
        decoder_optim.zero_grad()
        encoder_optim.zero_grad()
    loss_sum /= len(data)

    print(f"epoch {e+1} | loss {loss_sum}")

encoder.eval()
decoder.eval()
torch.save(encoder.to("cpu"), 'encoder.pt')
torch.save(decoder.to("cpu"), 'decoder.pt')

epoch 1 | loss 2.0441778058097477
epoch 2 | loss 1.7387495790209089
epoch 3 | loss 1.1542143509501503
epoch 4 | loss 0.6046182169800713
epoch 5 | loss 0.2637590237316631
epoch 6 | loss 0.1059524117303746
epoch 7 | loss 0.057401058685389306
epoch 8 | loss 0.019258661246636794
epoch 9 | loss 0.012682095127889799
epoch 10 | loss 0.008900907126787518
epoch 11 | loss 0.006524742715105059
epoch 12 | loss 0.0049787548331854245
epoch 13 | loss 0.003925150806372542
epoch 14 | loss 0.003178751379961059
epoch 15 | loss 0.002632137972839354
epoch 16 | loss 0.0022162297283232746
epoch 17 | loss 0.0018896977813537455
epoch 18 | loss 0.0016284974837963958
epoch 19 | loss 0.0014158057434196095
epoch 20 | loss 0.0012405425210350327


In [7]:
# 다시 불러와서 하나로 합치기
encoder = torch.load('encoder.pt', weights_only=False, map_location="cpu")
decoder = torch.load('decoder.pt', weights_only=False, map_location="cpu")

F = Encoder_n_Decoder(encoder, decoder)
F.eval()
torch.save(F.to("cpu"), "sound_ai.pt")

In [8]:
## AI 정확도 알아보기

correct = 0

for x, t in data :
    with torch.no_grad() : #자동 미분기능 끄는 함수, 안넣어도 문제는 없지만, 넣으면 살짝 더 빨라집니다.
        y = F(x)
    if y.argmax(dim = -1).item() == t :
        correct += 1
        
acc = correct / len(data)
print(acc)

1.0


In [9]:
# (문장갯수, 단어갯수, 벡터)
# (소리갯수, 소리길이, 벡터)
# (batchs, features, vector)
# (b, f, v) # batch first는 이렇게 자료를 넣겠다는 뜻이고

# (f, b, v) # batch first가 false일 경우